Taken from:
https://keras.io/examples/nlp/ner_transformers/

Introduction

Named Entity Recognition (NER) is the process of identifying named entities in text. Example of named entities are: "Person", "Location", "Organization", "Dates" etc. NER is essentially a token classification task where every token is classified into one or more predetermined categories.

In this exercise, we will train a simple Transformer based model to perform NER. We will be using the data from CoNLL 2003 shared task. For more information about the dataset, please visit the dataset website (https://www.clips.uantwerpen.be/conll2003/ner/). However, since obtaining this data requires an additional step of getting a free license, we will be using HuggingFace's datasets library which contains a processed version of this dataset.

In [1]:
# !pip3 install datasets
# !wget https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py


In [2]:
import os
import numpy as np
import tensorflow as tf
tf.load_library("/etc/alternatives/libcudnn_so")
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from datasets import load_dataset
from collections import Counter
from conlleval import evaluate
%load_ext autotime

2023-10-23 09:30:36.893308: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 09:30:36.893336: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 09:30:36.893357: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 09:30:36.898758: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 09:30:37.577843: W tensorflow/compiler/

time: 121 µs (started: 2023-10-23 09:30:38 -04:00)


/home/daniel/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# strategy = tf.distribute.MirroredStrategy()

# print('Number of devices: {}'.format(strategy.num_replicas_in_sync))


# Define your TensorFlow cluster configuration.
# Replace this with your actual cluster configuration.
cluster_spec = {
    "worker": ["worker1:port", "worker2:port"],
    "chief": ["chief1:port"]
}

cluster_resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver(cluster_spec)

# Create a MultiWorkerMirroredStrategy
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
    communication=tf.distribute.experimental.CollectiveCommunication.AUTO,
    cluster_resolver=cluster_resolver
)

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))


2023-10-23 09:30:38.435446: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-23 09:30:38.435633: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-23 09:30:38.456541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Number of devices: 2
time: 1 s (started: 2023-10-23 09:30:38 -04:00)


blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-23 09:30:38.581838: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-23 09:30:38.581977: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-23 09:30:39.305759: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github

In [4]:
train = 'Y'

time: 268 µs (started: 2023-10-23 09:30:39 -04:00)


In [5]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(ff_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


time: 1.24 ms (started: 2023-10-23 09:30:39 -04:00)


In [6]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, inputs):
        maxlen = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        position_embeddings = self.pos_emb(positions)
        token_embeddings = self.token_emb(inputs)
        return token_embeddings + position_embeddings


time: 3.55 ms (started: 2023-10-23 09:30:39 -04:00)


In [7]:
class NERModel(keras.Model):
    def __init__(
        self, num_tags, vocab_size, maxlen=128, embed_dim=32, num_heads=2, ff_dim=32
    ):
        super().__init__()
        self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.dropout1 = layers.Dropout(0.1)
        self.ff = layers.Dense(ff_dim, activation="relu")
        self.dropout2 = layers.Dropout(0.1)
        self.ff_final = layers.Dense(num_tags, activation="softmax")

    def call(self, inputs, training=False):
        x = self.embedding_layer(inputs)
        x = self.transformer_block(x)
        x = self.dropout1(x, training=training)
        x = self.ff(x)
        x = self.dropout2(x, training=training)
        x = self.ff_final(x)
        return x


time: 1.66 ms (started: 2023-10-23 09:30:39 -04:00)


In [8]:
conll_data = load_dataset("conll2003")


time: 5.77 s (started: 2023-10-23 09:30:39 -04:00)


In [9]:
def export_to_file_examine(export_file_path, data):
    with open(export_file_path, "w") as f:
        for record in data:
            tokens = record["tokens"]
            if len(tokens) > 0:
                f.write(
                    " ".join(tokens)
                    + "\n"
                )

# os.mkdir("data")
export_to_file_examine("./data/conll_examine.txt", conll_data["train"])

time: 793 ms (started: 2023-10-23 09:30:45 -04:00)


In [10]:
# def export_to_file(export_file_path, data):
#     with open(export_file_path, "w") as f:
#         for record in data:
#             ner_tags = record["ner_tags"]
#             tokens = record["tokens"]
#             if len(tokens) > 0:
#                 f.write(
#                     str(len(tokens))
#                     + "\t"
#                     + "\t".join(tokens)
#                     + "\t"
#                     + "\t".join(map(str, ner_tags))
#                     + "\n"
#                 )


# # os.mkdir("data")
# export_to_file("./data/conll_train.txt", conll_data["train"])
# export_to_file("./data/conll_val.txt", conll_data["validation"])


time: 328 µs (started: 2023-10-23 09:30:46 -04:00)


In [11]:
def export_to_file(export_file_path, data):
    with open(export_file_path, "w") as f:
        for record in data:
            ner_tags = record["ner_tags"]
            tokens = record["tokens"]
            if len(tokens) > 0:
                binary_tags = per_tags(ner_tags)
                f.write(
                    str(len(tokens))
                    + "\t"
                    + "\t".join(tokens)
                    + "\t"
                    + "\t".join(map(str, binary_tags))
                    + "\n"
                )

def per_tags(nertags):
    return [1 if x in (1,2) else 0 for x in nertags]

# Uncomment this if you want to create the directory
# os.mkdir("data")
export_to_file("./data/conll_train.txt", conll_data["train"])
export_to_file("./data/conll_val.txt", conll_data["validation"])

time: 1.01 s (started: 2023-10-23 09:30:46 -04:00)


In [12]:
def make_tag_lookup_table():
    iob_labels = ["B", "I"]
    ner_labels = ["PER", "ORG", "LOC", "MISC"]
    all_labels = [(label1, label2) for label2 in ner_labels for label1 in iob_labels]
    all_labels = ["-".join([a, b]) for a, b in all_labels]
    all_labels = ["[PAD]", "O"] + all_labels
    return dict(zip(range(0, len(all_labels) + 1), all_labels))


mapping = make_tag_lookup_table()
print(mapping)


{0: '[PAD]', 1: 'O', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}
time: 657 µs (started: 2023-10-23 09:30:47 -04:00)


In [13]:
all_tokens = sum(conll_data["train"]["tokens"], [])
all_tokens_array = np.array(list(map(str.lower, all_tokens)))

counter = Counter(all_tokens_array)
print(len(counter))

num_tags = len(mapping)
vocab_size = 20000

# We only take (vocab_size - 2) most commons words from the training data since
# the `StringLookup` class uses 2 additional tokens - one denoting an unknown
# token and another one denoting a masking token
vocabulary = [token for token, count in counter.most_common(vocab_size - 2)]

# The StringLook class will convert tokens to token IDs
lookup_layer = keras.layers.StringLookup(
    vocabulary=vocabulary
)


21009
time: 5.75 s (started: 2023-10-23 09:30:47 -04:00)


In [14]:
train_data = tf.data.TextLineDataset("./data/conll_train.txt")
val_data = tf.data.TextLineDataset("./data/conll_val.txt")


time: 14.5 ms (started: 2023-10-23 09:30:52 -04:00)


In [15]:
print(list(train_data.take(1).as_numpy_iterator()))


[b'9\tEU\trejects\tGerman\tcall\tto\tboycott\tBritish\tlamb\t.\t0\t0\t0\t0\t0\t0\t0\t0\t0']
time: 42 ms (started: 2023-10-23 09:30:52 -04:00)


In [16]:
def map_record_to_training_data(record):
    record = tf.strings.split(record, sep="\t")
    length = tf.strings.to_number(record[0], out_type=tf.int32)
    tokens = record[1 : length + 1]
    tags = record[length + 1 :]
    tags = tf.strings.to_number(tags, out_type=tf.int64)
    tags += 1
    return tokens, tags


def lowercase_and_convert_to_ids(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)


# We use `padded_batch` here because each record in the dataset has a
# different length.
batch_size = 32
train_dataset = (
    train_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)
val_dataset = (
    val_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)


time: 166 ms (started: 2023-10-23 09:30:52 -04:00)


In [17]:
class CustomNonPaddingTokenLoss(keras.losses.Loss):
    def __init__(self, name="custom_ner_loss"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        loss_fn = keras.losses.SparseCategoricalCrossentropy(
            from_logits=False, reduction=keras.losses.Reduction.NONE
        )
        loss = loss_fn(y_true, y_pred)
        mask = tf.cast((y_true > 0), dtype=tf.float32)
        loss = loss * mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)


loss = CustomNonPaddingTokenLoss()


time: 527 µs (started: 2023-10-23 09:30:53 -04:00)


In [18]:
with strategy.scope():
    es = EarlyStopping(monitor='val_accuracy', mode='min', verbose=2, patience=5)
    ner_model = NERModel(num_tags, vocab_size, embed_dim=32, num_heads=4, ff_dim=64)
    history = ner_model.compile(optimizer="adam", loss=loss, metrics=['accuracy'])

time: 39.5 ms (started: 2023-10-23 09:30:53 -04:00)


In [19]:
class CustomSaveModel(tf.keras.callbacks.Callback):
    def __init__(self, save_freq):
        super(CustomSaveModel, self).__init__()
        self.save_freq = save_freq
    
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            save_path = f"ner_saved_{epoch + 1}_20231023"
            self.model.save(save_path)
            print(f"Model saved at epoch: {epoch + 1}")

if train == 'Y':
    save_callback = CustomSaveModel(save_freq=250)
    ner_model.fit(train_dataset, epochs=500, callbacks=[save_callback])

Epoch 1/500


2023-10-23 09:30:58.778027: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc6c80cdb50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-23 09:30:58.778051: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-10-23 09:30:58.778058: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-10-23 09:30:58.785177: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-23 09:30:58.910531: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-10-23 09:30:58.979867: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-10-23 09:30:59.

439/439 [==============================] - 38s 72ms/step - loss: 0.2156 - accuracy: 0.3871
Epoch 2/500
  1/439 [..............................] - ETA: 5s - loss: 0.0973 - accuracy: 0.4781

2023-10-23 09:31:31.490874: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13653882349482229730
2023-10-23 09:31:31.490929: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10078033694558757461
2023-10-23 09:31:31.490935: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10024542798458750951
2023-10-23 09:31:31.490940: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1310130686640892034
2023-10-23 09:31:31.490945: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8051083902128466386
2023-10-23 09:31:31.490949: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4512108454896405146
2023-10-23 09:31:31.490953: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv 

439/439 [==============================] - 5s 11ms/step - loss: 0.0405 - accuracy: 0.4041
Epoch 3/500
439/439 [==============================] - 5s 11ms/step - loss: 0.0137 - accuracy: 0.4073
Epoch 4/500
439/439 [==============================] - 5s 11ms/step - loss: 0.0085 - accuracy: 0.4078
Epoch 5/500
439/439 [==============================] - 5s 11ms/step - loss: 0.0061 - accuracy: 0.4082
Epoch 6/500
439/439 [==============================] - 5s 11ms/step - loss: 0.0067 - accuracy: 0.4080
Epoch 7/500
439/439 [==============================] - 5s 11ms/step - loss: 0.0048 - accuracy: 0.4083
Epoch 8/500
439/439 [==============================] - 5s 11ms/step - loss: 0.0036 - accuracy: 0.4085
Epoch 9/500
439/439 [==============================] - 5s 11ms/step - loss: 0.0029 - accuracy: 0.4085
Epoch 10/500
439/439 [==============================] - 5s 11ms/step - loss: 0.0031 - accuracy: 0.4085
Epoch 11/500
439/439 [==============================] - 5s 11ms/step - loss: 0.0027 - accurac

In [27]:


def tokenize_and_convert_to_ids(text):
    tokens = text.split()
    return lowercase_and_convert_to_ids(tokens)


# Sample inference using the trained model
sample_input = tokenize_and_convert_to_ids(
    "eu rejects german call to boycott british lamb from Steve parson the funky Parson"
)
sample_input = tf.reshape(sample_input, shape=[1, -1])
print(sample_input)

output = ner_model.predict(sample_input)
prediction = np.argmax(output, axis=-1)[0]
prediction = [mapping[i] for i in prediction]

# eu -> B-ORG, german -> B-MISC, british -> B-MISC
print(prediction)



tf.Tensor(
[[  988 10950   204   628     6  3938   215  5773    26  1036     0     1
      0     0]], shape=(1, 14), dtype=int64)
1/1 [==============================] - 0s 106ms/step
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'B-PER', 'O', 'O', 'O']
time: 182 ms (started: 2023-10-23 13:45:29 -04:00)


In [26]:
def calculate_metrics(dataset):
    all_true_tag_ids, all_predicted_tag_ids = [], []

    for x, y in dataset:
        output = ner_model.predict(x)
        predictions = np.argmax(output, axis=-1)
        predictions = np.reshape(predictions, [-1])

        true_tag_ids = np.reshape(y, [-1])

        mask = (true_tag_ids > 0) & (predictions > 0)
        true_tag_ids = true_tag_ids[mask]
        predicted_tag_ids = predictions[mask]

        all_true_tag_ids.append(true_tag_ids)
        all_predicted_tag_ids.append(predicted_tag_ids)

    all_true_tag_ids = np.concatenate(all_true_tag_ids)
    all_predicted_tag_ids = np.concatenate(all_predicted_tag_ids)

    predicted_tags = [mapping[tag] for tag in all_predicted_tag_ids]
    real_tags = [mapping[tag] for tag in all_true_tag_ids]

    evaluate(real_tags, predicted_tags)


calculate_metrics(val_dataset)


1/1 [==============================] - 0s 86ms/step
processed 51362 tokens with 3149 phrases; found: 2685 phrases; correct: 2344.
accuracy:  74.44%; (non-O)
accuracy:  97.77%; precision:  87.30%; recall:  74.44%; FB1:  80.36
              PER: precision:  87.30%; recall:  74.44%; FB1:  80.36  2685
time: 13 s (started: 2023-10-23 12:55:37 -04:00)
